In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

USE_MPS = True

# device
if USE_MPS:
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("🍎 Using Apple MPS GPU")
    else:
        device = torch.device("cpu")
        print("💻 Using CPU")
else:
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("🔥 Using NVIDIA CUDA GPU")
    else:
        device = torch.device("cpu")
        print("💻 Using CPU")


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_ds = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)

test_ds = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)


In [ ]:
import numpy as np

x, y = next(iter(train_loader))

plt.figure(figsize=(8,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(x[i][0], cmap="gray")
    plt.title(str(y[i].item()))
    plt.axis("off")
plt.show()


## CNN 訓練

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(9216, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

model = CNN().to(device)
model


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
train_losses = []

EPOCHES = 15

for epoch in range(EPOCHES):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg = total_loss / len(train_loader)
    train_losses.append(avg)
    print(f"Epoch {epoch+1} | loss = {avg:.4f}")


In [ ]:
# Plot training loss

plt.plot(train_losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.show()


In [ ]:
model.eval()
correct = 0

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(dim=1)
        correct += (pred == y).sum().item()

acc = correct / len(test_ds)
print("Test accuracy =", acc)


## 儲存模型

In [ ]:
# 建議路徑後綴使用 .pth 或 .pt
model_path = "mnist_cnn.pth"

# 儲存模型權重 (state_dict)
torch.save(model.state_dict(), model_path)
print(f"模型已儲存至 {model_path}")

## 讀取模型

In [ ]:
LOAD_MODEL = False

if LOAD_MODEL:
    # 1. 初始化模型結構 (必須與訓練時定義的 CNN 類一致)
    loaded_model = CNN() 

    # 2. 載入權重檔案
    model_path = "mnist_cnn.pth"
    state_dict = torch.load(model_path, map_location=device) # 考慮到可能在不同設備間載入

    # 3. 將權重套用到模型中
    loaded_model.load_state_dict(state_dict)

    # 4. 將模型移至正確的設備 (CPU 或 MPS/GPU) 並設為評估模式
    # 如果有使用 MPS/GPU，記得把 model.to(device) 設定成正確的設備

    loaded_model.to(device)
    loaded_model.eval()

    print("模型載入成功，可以直接進行預測。")

## User 輸入

In [ ]:
from PIL import Image
import torchvision.transforms.functional as F


img_path = "5.1.png"   # 放你自己的圖

img = Image.open(img_path).convert("L")  # 轉灰階
plt.imshow(img, cmap="gray")
plt.axis("off")


In [ ]:
# 如果圖是白底黑字，記得反轉
img = F.invert(img)

In [ ]:
import numpy as np

arr = np.array(img)

# 自動判斷前景顏色
if arr.mean() > 127:   # 白底黑字
    mask = arr < 200
else:                  # 黑底白字
    mask = arr > 50

ys, xs = np.where(mask)

top, bottom = ys.min(), ys.max()
left, right = xs.min(), xs.max()

img_crop = img.crop((left, top, right, bottom))
plt.imshow(img_crop, cmap="gray")
plt.axis("off")


In [ ]:
IS_CROP = False

import torchvision.transforms as T
transform = T.Compose([
    T.Resize((28,28)),      # ⚠️ 關鍵
    T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,))
])

if IS_CROP:
    x = transform(img_crop)
else:
    x = transform(img)
x = x.unsqueeze(0).to(device)

plt.imshow(x[0][0].cpu(), cmap="gray")
plt.title("Final 28x28")
plt.axis("off")


In [ ]:
model.eval()

with torch.no_grad():
    out = model(x)
    pred = out.argmax(dim=1).item()

print("模型猜的是:", pred)